Create a real-time inferencing ML Service 
(implemented with Azure Python SDK in VS Code)

Before running this notebook, make sure that 
- Python 3.7.8 is installed (some troubles can appear with later editions)
- Install following VS Code Extensions: Python(IntelliSence(Pylance)), Azure Tools, Azure Machine Learning

- Install following packages in powershell: 
pip install pandas
pip install joblib
pip install scikit-learn
pip install azureml-core
pip install azure-ai-ml
pip install azureml-dataprep
pip install azureml-widgets


In [1]:
# Creating workspace
import azureml.core
from azureml.core import Workspace

ws = Workspace.create(name='aml-workspace', 
                    subscription_id='1d702fb9-2f61-4de9-94be-1fff9c158887',
                    resource_group='aml-resources',
                    create_resource_group=True,
                    location='eastus')


Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (importlib-metadata 5.1.0 (c:\users\winpr\appdata\local\programs\python\python37\lib\site-packages), Requirement.parse('importlib-metadata<5,>=0.23; python_version == "3.7"'), {'argcomplete'}).


Deploying AppInsights with name amlworksinsights54256b10.
Deployed AppInsights with name amlworksinsights54256b10. Took 5.08 seconds.
Deploying KeyVault with name amlworkskeyvaultf1fa1ae2.
Deploying StorageAccount with name amlworksstorage7a0082bc4.
Deployed KeyVault with name amlworkskeyvaultf1fa1ae2. Took 20.31 seconds.
Deployed StorageAccount with name amlworksstorage7a0082bc4. Took 26.24 seconds.
Deploying Workspace with name aml-workspace.
Deployed Workspace with name aml-workspace. Took 24.97 seconds.


In [2]:
#check if created and ready to use
print("Ready to use Azure ML {} to work with {}".format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with aml-workspace


In [3]:
# use the following code to enumerate the compute resources in your workspace
print("Compute Resources:")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)

Compute Resources:


Create compute target
We'll need a compute context for the pipeline, so we'll use the following code to specify an Azure Machine Learning compute cluster. Cluster names must be globally unique names between 2 to 16 characters in length. Valid characters are letters, digits, and the - character.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-cluster111"
try: 
    #check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    #if it doesn't already exist, create it
    try: 
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        inference_cluster=ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
#check available compute targets again
print("Compute Resources:")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)

Compute Resources:
	 compute-cluster111 : AmlCompute


In [6]:
# Upload the data to datastore (remember Datastore - is just a connection to your real data)
from azureml.core import Dataset
from azureml.data.datapath import DataPath

default_ds= ws.get_default_datastore()
Dataset.File.upload_directory(src_dir='C:/Users/winpr/AzureML_Classification_PythonSDK/data', target=DataPath(default_ds, 'diabetes-data/'))

Validating arguments.
Arguments validated.
Uploading file to diabetes-data/
Uploading an estimated of 2 files
Uploading C:/Users/winpr/AzureML_Classification_PythonSDK/data\diabetes2.csv
Uploaded C:/Users/winpr/AzureML_Classification_PythonSDK/data\diabetes2.csv, 1 files out of an estimated total of 2
Uploading C:/Users/winpr/AzureML_Classification_PythonSDK/data\diabetes.csv
Uploaded C:/Users/winpr/AzureML_Classification_PythonSDK/data\diabetes.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/diabetes-data/')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ]
}

In [7]:
# create a tabular dataset
from azureml.core import Dataset
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'), header=True)

# display the first 20 rows
tab_data_set.take(5).to_pandas_dataframe()

c:\Users\winpr\AppData\Local\Programs\Python\Python37\lib\site-packages\azureml\dataprep\api\_dataframereader.py:461: UserWarning: Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.16.0 --upgrade
  "Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. "


,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0


In [8]:
# register dataset in AzureML Workspace
try:
    tab_data_set = tab_data_set.register(workspace=ws,name= 'diabetes file dataset',description='diabetes files',tags={'format':'CSV'}, create_new_version=True)
except Exception as ex:
    print(ex)

Here to avoid snapshot error had to configure experiment.start_logging(snapshot_directory=None)
https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py

In [9]:
# Train and register the sklearn DecisionTreeClassifier model
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd 
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#Create AzureML experiment in your workspace
experiment= Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging(snapshot_directory=None) # to avoid snapshot error
print("Starting experiment:", experiment.name)

#Load the diabetes dataset
print("Loading Data...")
#diabetes = pd.read_csv(tab_data_set)
diabetes = tab_data_set.to_pandas_dataframe()

#Separate features and labels
X,y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=0)

#Train a desicion tree model
print("Training a decision tree model")
model = DecisionTreeClassifier().fit(X_train, y_train)

#calculate accuracy
y_hat= model.predict(X_test)
acc= np.average(y_hat==y_test)
print("Accuracy:", acc)
run.log('Accuracy', np.float(acc))

#calculate AUC
y_scores= model.predict_proba(X_test)
auc= roc_auc_score(y_test, y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

#Save the trained model. The best option to keep all the data in project directory
model_file= 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name='AzureML_Classification_PythonSDK/outputs/' + model_file, path_or_stream= './' + model_file)

#Complete the run
run.complete()

#Register the model
run.register_model(model_path='AzureML_Classification_PythonSDK/outputs/diabetes_model.pkl', model_name='diabetes_model',tags={'Training context':'Inline Training'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print("Model trained and registered")

Starting experiment: mslearn-train-diabetes
Loading Data...


c:\Users\winpr\AppData\Local\Programs\Python\Python37\lib\site-packages\azureml\dataprep\api\_dataframereader.py:461: UserWarning: Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.16.0 --upgrade
  "Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. "


Training a decision tree model
Accuracy: 0.8966666666666666
AUC: 0.8812731098273671


C:\Users\winpr\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:36: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\winpr\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:42: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Model trained and registered


Deploy the model as a web service

In [10]:
# loop through available models

from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag= model.tags[tag_name]
        print('\t', tag_name, ':', tag)
    for prop_name in model.properties:
        prop=model.properties[prop_name]
        print('\t', prop_name, ':', prop)
    print('\n')

diabetes_model version: 1
	 Training context : Inline Training
	 AUC : 0.8812731098273671
	 Accuracy : 0.8966666666666666




In [11]:
# et the model that we want to deploy
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

diabetes_model version 1


We're going to create a web service to host this model, and this will require some code and configuration files; so let's create a folder for those.

In [12]:
import os
# Create a folder for the deployment files
#deployment_folder= './diabetes_service'  - to deploy in default wd
#deploy to project directory : 

# deploy to project folder 'C:/Users/winpr/AzureML_Classification_PythonSDK/
deployment_folder= 'AzureML_Classification_PythonSDK/diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder, script_file)


AzureML_Classification_PythonSDK/diabetes_service folder created.


The web service where we deploy the model will need some Python code to load the input data, get the model from the workspace, and generate and return predictions. We'll save this code in an entry script (often called a scoring script) that will be deployed to the web service.

remember to place %% magic functions at the begining of code cell (no #comments before %% allowed)

In [13]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

#Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

#Called when a request is received
def run(raw_data):
    #Get the input data as a numpy array
    data= np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing AzureML_Classification_PythonSDK/diabetes_service\score_diabetes.py


The web service will be hosted in a container, and the container will need to install any required Python dependencies when it gets initialized. In this case, our scoring code requires scikit-learn and some Azure Machine Learning specific packages that are used by the scoring web service, so we'll create an environment that included these. Then we'll add that environment to an inference configuration along with the scoring script, and define a deployment configuration for the container in which the environment and script will be hosted.
We can then deploy the model as a service based on these configurations.

    More Information: For more details about model deployment, and options for target execution environments, see the documentation.
    https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?tabs=azure-cli

In [14]:
# Deployment can take up to 30 min to create a container image, and then runs a process to create a web service 
# based on the image. 
# When deployment has completed successfully, you'll see a status of Healthy.

from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
# here "AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference" - specific configuration
service_env= Environment.get(workspace=ws, name= "AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
service_env.inferencing_stack_version= "latest"

inference_config = InferenceConfig(source_directory = deployment_folder, entry_script=script_file, environment = service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb =1)

# Deploy the model as a service
print ('Deploying model...')
service_name = "diabetes-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite = True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...


C:\Users\winpr\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-20 11:14:11+01:00 Creating Container Registry if not exists..
2022-12-20 11:24:12+01:00 Registering the environment.
2022-12-20 11:24:13+01:00 Use the existing image.
2022-12-20 11:24:13+01:00 Generating deployment configuration.
2022-12-20 11:24:14+01:00 Submitting deployment to compute..
2022-12-20 11:24:19+01:00 Checking the status of deployment diabetes-service..
2022-12-20 11:25:52+01:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [15]:
# Hopefully, the deployment has been successful and you can see a status of Healthy. 
# If not, you can use the following code to get the service logs to help you troubleshoot.
print(service.get_logs())

2022-12-20T10:25:46,408621200+00:00 - nginx/run 
2022-12-20T10:25:46,409664400+00:00 - gunicorn/run 
2022-12-20T10:25:46,415225000+00:00 | gunicorn/run | 
2022-12-20T10:25:46,417288600+00:00 | gunicorn/run | ###############################################
2022-12-20T10:25:46,423421200+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-12-20T10:25:46,426348100+00:00 | gunicorn/run | ###############################################
2022-12-20T10:25:46,430914000+00:00 | gunicorn/run | 
2022-12-20T10:25:46,433430200+00:00 | gunicorn/run | 
2022-12-20T10:25:46,442497500+00:00 | gunicorn/run | AzureML image information: sklearn-0.24.1-ubuntu18.04-py37-cpu-inference:20221024.v1
2022-12-20T10:25:46,446803900+00:00 | gunicorn/run | 
2022-12-20T10:25:46,448483200+00:00 | gunicorn/run | 
2022-12-20T10:25:46,449975500+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/amlenv/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
2022-12

In [16]:
# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

In [17]:
# You can also retrieve the names of web services in your workspace by running the following code:
for webservice_name in ws.webservices:
    print(webservice_name)

diabetes-service


Use the web service

With the service deployed, now you can consume it from a client application to make a prediction on the new data provided to the model

In [18]:
import json

# provide new feature data
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
diabetic


In [19]:
# You can also send multiple patient observations to the service, and get back a prediction for each one.
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


The code above uses the Azure Machine Learning SDK to connect to the containerized web service and use it to generate predictions from your diabetes classification model. In production, a model is likely to be consumed by business applications that do not use the Azure Machine Learning SDK, but simply make HTTP requests to the web service.

In [20]:
#Let's determine the URL to which these applications must submit their requests:
endpoint = service.scoring_uri
print(endpoint)

http://2a2b6b15-2484-4e94-8300-369f8b087c19.eastus.azurecontainer.io/score


Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON format, and receive back the predicted class(es).

In [21]:
# make HTTP request containing new data for prediction
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


You've deployed your web service as an Azure Container Instance (ACI) service that requires no authentication. This is fine for development and testing, but for production you should consider deploying to an Azure Kubernetes Service (AKS) cluster and enabling token-based authentication. This would require REST requests to include an Authorization header.

Delete the service to avoid incurring unecessary charges.

service.delete()
print ('Service deleted.')